In [1]:
import pickle as pk
import os
import numpy as np
import pandas as pd

In [2]:
transactions_df = None

In [3]:
directory = 'simulated-data-raw/'

for file in os.listdir(directory):
    if file.endswith('.pkl') and file.startswith('2018'):
        filepath = os.path.join(directory, file)
        with open(filepath, 'rb') as f:
            data = pk.load(f)
        if transactions_df is None:
            transactions_df = data
        else:
            transactions_df = pd.concat([transactions_df, data], ignore_index=True)


#pk.load(open('simulated-data-raw/2018-09-22.pkl', 'rb'))

In [4]:
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,0,2018-04-01 00:00:11,19752,35415,41.55,11,0,0,0
1,1,2018-04-01 00:00:17,6160,11302,31.83,17,0,0,0
2,2,2018-04-01 00:00:28,14208,15475,73.08,28,0,0,0
3,3,2018-04-01 00:00:31,596,15978,57.16,31,0,0,0
4,4,2018-04-01 00:00:32,11839,15655,136.62,32,0,0,0


In [5]:
# Rename columns
transactions_df.rename(columns={
    'CUSTOMER_ID': 'card_id' ,
    'TERMINAL_ID': 'terminal_id',
    'TX_FRAUD': 'is_fraud',
    'TX_AMOUNT': 'amount',
    # 'TX_TIME_DAYS': 'day_index',
    'TX_DATETIME': 'timestamp',
}, inplace=True)

transactions_df.drop(['TX_FRAUD_SCENARIO', 'TRANSACTION_ID', 'TX_TIME_SECONDS', 'TX_TIME_DAYS'], axis=1, inplace=True)
transactions_df['is_online'] = np.zeros(transactions_df.shape[0])

In [6]:
transactions_df.head()

,timestamp,card_id,terminal_id,amount,is_fraud,is_online
0,2018-04-01 00:00:11,19752,35415,41.55,0,0.0
1,2018-04-01 00:00:17,6160,11302,31.83,0,0.0
2,2018-04-01 00:00:28,14208,15475,73.08,0,0.0
3,2018-04-01 00:00:31,596,15978,57.16,0,0.0
4,2018-04-01 00:00:32,11839,15655,136.62,0,0.0


In [7]:
import datetime

transactions_df['timestamp'] = transactions_df['timestamp'].dt.to_pydatetime()



C:\Users\dalun\AppData\Local\Temp\ipykernel_2716\2611337306.py:3: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  transactions_df['timestamp'] = transactions_df['timestamp'].dt.to_pydatetime()


In [8]:
with open('simulated-data-raw/customer_profiles_table.pkl', 'rb') as f:
    customer_profiles_table = pk.load(f)

with open('simulated-data-raw/terminal_profiles_table.pkl', 'rb') as f:
    terminal_profiles_table = pk.load(f)

In [9]:
customer_profiles_table

,CUSTOMER_ID,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,available_terminals,nb_terminals
0,0,109.762701,143.037873,62.262521,31.131260,2.179533,"[1067, 1536, 1699, 2007, 3028, 3246, 3457, 367...",77
1,1,84.730960,129.178823,46.570785,23.285393,3.567092,"[2443, 2641, 2650, 2859, 3258, 5159, 5385, 567...",74
2,2,192.732552,76.688304,80.213879,40.106939,2.115580,"[447, 1318, 2149, 2374, 2586, 3598, 3876, 4385...",65
3,3,113.608912,185.119328,11.748426,5.874213,0.348517,"[364, 433, 772, 925, 1924, 2338, 3710, 4077, 4...",98
4,4,4.043679,166.523969,78.924891,39.462446,3.480049,"[372, 614, 1362, 1564, 1637, 1939, 3205, 3279,...",88
...,...,...,...,...,...,...,...,...
19995,19995,124.291148,101.500964,51.721364,25.860682,2.192022,"[532, 1994, 2895, 3002, 3809, 4113, 4567, 4630...",78
19996,19996,169.471429,67.720102,13.779435,6.889718,1.891497,"[132, 773, 867, 1256, 1293, 2781, 3391, 3908, ...",89
19997,19997,2.785287,126.877808,77.706479,38.853239,1.475556,"[49, 1787, 2113, 2972, 3249, 3764, 4527, 8199,...",59
19998,19998,85.093332,135.571292,23.030211,11.515105,0.553780,"[5, 378, 571, 943, 1460, 2443, 2459, 2828, 325...",82


In [10]:
customer_profiles_table['x_customer_id'] = np.round(customer_profiles_table['x_customer_id'])
customer_profiles_table['y_customer_id'] = np.round(customer_profiles_table['y_customer_id'])

customer_profiles_table['balance'] = (customer_profiles_table['mean_amount'] * customer_profiles_table['mean_nb_tx_per_day'] * 60).round(2)

customer_profiles_table.head()

,CUSTOMER_ID,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,available_terminals,nb_terminals,balance
0,0,110.0,143.0,62.262521,31.131260,2.179533,"[1067, 1536, 1699, 2007, 3028, 3246, 3457, 367...",77,8142.19
1,1,85.0,129.0,46.570785,23.285393,3.567092,"[2443, 2641, 2650, 2859, 3258, 5159, 5385, 567...",74,9967.34
2,2,193.0,77.0,80.213879,40.106939,2.115580,"[447, 1318, 2149, 2374, 2586, 3598, 3876, 4385...",65,10181.93
3,3,114.0,185.0,11.748426,5.874213,0.348517,"[364, 433, 772, 925, 1924, 2338, 3710, 4077, 4...",98,245.67
4,4,4.0,167.0,78.924891,39.462446,3.480049,"[372, 614, 1362, 1564, 1637, 1939, 3205, 3279,...",88,16479.75


In [11]:
terminal_profiles_table['x_terminal_id'] = np.round(terminal_profiles_table['x_terminal_id'])
terminal_profiles_table['y_terminal_id'] = np.round(terminal_profiles_table['y_terminal_id'])

terminal_profiles_table.head()

,TERMINAL_ID,x_terminal_id,y_terminal_id
0,0,83.0,144.0
1,1,0.0,60.0
2,2,29.0,18.0
3,3,37.0,69.0
4,4,79.0,108.0


In [12]:
customer_profiles_table.rename(columns={
    'CUSTOMER_ID': 'id',
    'x_customer_id': 'x',
    'y_customer_id': 'y',
}, inplace=True)

terminal_profiles_table.rename(columns={
    'TERMINAL_ID': 'id',
    'x_terminal_id': 'x',
    'y_terminal_id': 'y',
}, inplace=True)

customer_profiles_table = customer_profiles_table[['id', 'x', 'y', 'balance']]
terminal_profiles_table = terminal_profiles_table[['id', 'x', 'y']]

In [13]:
# Save the processed dataframes
transactions_df.to_csv('transactions.csv', index=False)
customer_profiles_table.to_csv('customer_profiles.csv', index=False)
terminal_profiles_table.to_csv('terminal_profiles.csv', index=False)

In [14]:
transactions_df.head()

,timestamp,card_id,terminal_id,amount,is_fraud,is_online
0,2018-04-01 00:00:11,19752,35415,41.55,0,0.0
1,2018-04-01 00:00:17,6160,11302,31.83,0,0.0
2,2018-04-01 00:00:28,14208,15475,73.08,0,0.0
3,2018-04-01 00:00:31,596,15978,57.16,0,0.0
4,2018-04-01 00:00:32,11839,15655,136.62,0,0.0


In [15]:
customer_profiles_table.head()

,id,x,y,balance
0,0,110.0,143.0,8142.19
1,1,85.0,129.0,9967.34
2,2,193.0,77.0,10181.93
3,3,114.0,185.0,245.67
4,4,4.0,167.0,16479.75


In [16]:
max(transactions_df['card_id']), max(customer_profiles_table['id']), max(transactions_df['terminal_id']), max(terminal_profiles_table['id'])

(19999, 19999, 39999, 39999)